# Fake Structured Content Generation

Using a customized prompt, along with a few shot based prompting method to generate stuctured fake content for testing stuff out effectively

In [2]:
from getpass import getpass
GOOGLE_API_KEY = getpass()

 ········


In [3]:
from langchain_core.globals import set_llm_cache
from langchain_google_genai import GoogleGenerativeAI
from langchain_core.caches import InMemoryCache

llm = GoogleGenerativeAI(model="gemini-2.0-flash", google_api_key=GOOGLE_API_KEY)

set_llm_cache(InMemoryCache())

In [37]:
raw_prompt = """You can an advanced fake data generator and you excel in generating and
filling out fake data in whatever structured format that is given to you.
Your job now is to generate fake data based on for the given json format.
You will get an input in this format number | input | schema
You need to generate [number] examples based on this [input] in this schema [schema]"""

In [46]:
example = {
    "book_id": "journey-to-center",
    "author": "Jules Vern",
    "title": "The Journey to the center of the Earth",
    "description": "Awesome Book about exploration"
}

json_schema = {
    "book_id": "The Book ID",
    "author": "The Author of the book",
    "title": "The Book Title",
    "description": "A Description of the book", 
}

def make_prompt(number, example, schema):
    return f"{number} | {example} | {schema}"

import json
prompt = make_prompt(40, json.dumps(example), json.dumps(json_schema))

In [47]:
from google import genai
from google.genai import types

client = genai.Client(api_key=GOOGLE_API_KEY)
response = client.models.generate_content(
            model="gemini-2.0-flash",
            contents=[prompt],
            config=types.GenerateContentConfig(
                system_instruction=raw_prompt,
                temperature=1,
                response_mime_type="application/json",
            ),
        )

res = response.candidates[0].content.parts[0].text
print(res)

[
  {
    "book_id": "the-call-of-the-wild",
    "author": "Jack London",
    "title": "The Call of the Wild",
    "description": "Buck, a domesticated dog, is stolen from his home and sold into service as a sled dog in the Klondike."
  },
  {
    "book_id": "white-fang",
    "author": "Jack London",
    "title": "White Fang",
    "description": "White Fang is a wild wolfdog who gradually becomes domesticated."
  },
  {
    "book_id": "moby-dick",
    "author": "Herman Melville",
    "title": "Moby Dick",
    "description": "The story of Captain Ahab's obsessive quest to hunt the white whale, Moby Dick."
  },
  {
    "book_id": "pride-and-prejudice",
    "author": "Jane Austen",
    "title": "Pride and Prejudice",
    "description": "A novel about the Bennet sisters and their experiences with love, class, and society in 19th century England."
  },
  {
    "book_id": "jane-eyre",
    "author": "Charlotte Brontë",
    "title": "Jane Eyre",
    "description": "The story of Jane Eyre's jou

In [48]:
obj = json.loads(res)
print(len(obj))

40


In [52]:
with open("books.json", "w") as file:
    file.write(json.dumps({"books": obj}))